In [1]:
# model serialization

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [2]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [3]:
chain = prompt | llm 

chain.is_lc_serializable()

True

In [4]:
from langchain_core.load import dumpd, dumps

dumped_chain = dumpd(chain) 
dumped_chain # type - dict

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [5]:
dumps_chain = dumps(chain)
dumps_chain # type - str

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "max_retries": 2, "n": 1}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

In [6]:
import pickle

file_path = "sample_chain.pkl"
with open(file_path, "wb") as file:
    pickle.dump(dumped_chain, file)


In [8]:
with open(file_path, 'rb') as file:
    loaded_chain = pickle.load(file)

In [9]:
import os
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={'OPENAI_API_KEY': os.environ["OPENAI_API_KEY"]}
)

load_chain.invoke({'fruit': '사과'})

/tmp/ipykernel_3209215/405043039.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  load_chain = load(


AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e1ae5f22-7380-4196-a3e2-4d97e395e450-0', usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# huggingface endpoint

from dotenv import load_dotenv

load_dotenv()

from huggingface_hub import login

login()

In [15]:
from langchain.prompts import PromptTemplate

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='<|system|>\nYou are a helpful assistant.<|end|>\n<|user|>\n{question}<|end|>\n<|assistant|>')

In [18]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint 

repo_id = "microsoft/Phi-3-mini-4k-instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=256,
    temperature=0.1,
    huggingface_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"question": "What is the capital of the South Korea?"})
print(response)

WARNING! huggingface_api_token is not default parameter.
                    huggingface_api_token was transferred to model_kwargs.
                    Please make sure that huggingface_api_token is what you intended.


The capital of South Korea is Seoul. Seoul is not only the capital but also the largest metropolis in South Korea. It is a bustling city known for its modern skyscrapers, high-tech subways, and pop culture, as well as its historical sites such as palaces, temples, and traditional markets.


In [20]:
from langchain_core.prompts import ChatPromptTemplate

hf_endpoint_url = "https://slcalzucia3n7y3g.us-east-1.aws.endpoints.huggingface.cloud"

llm = HuggingFaceEndpoint(
    endpoint_url=hf_endpoint_url,
    max_new_tokens=256,
    temperature=0.1,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "A chat between a curios user and an aritificial intelligence assistant. The assistant gives helpful, detailed, and polite answer to the user's question",
        ),
        ("user", "Human: {question}:\nAssistant: ")
    ]
)

chain = prompt | llm | StrOutputParser()

chain.invoke("대한민국의 수도는 어디인가요?")

HfHubHTTPError: 404 Client Error: Not Found for url: https://slcalzucia3n7y3g.us-east-1.aws.endpoints.huggingface.cloud/ (Request ID: coCVmu)

Not Found: slcalzucia3n7y3g.us-east-1.aws.endpoints.huggingface.cloud